In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("GPU available:", gpus)
else:
    print("No GPU available.")


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install tensorflow
!pip install onnx-tf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 62.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from onnx_tf.backend import prepare


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8l-cls.pt')  # load an official model
model = YOLO('/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='onnx')

100%|██████████| 71.7M/71.7M [00:00<00:00, 270MB/s]


Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (Intel Xeon 2.00GHz)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.pt' with input shape (1, 3, 320, 320) BCHW and output shape(s) (1, 10) (69.2 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 2.5s, saved as '/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.onnx' (138.1 MB)

Export complete (5.4s)
Results saved to /content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights
Predict:         yolo predict task=classify model=/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.onnx imgsz=320  
Validate:        yolo val task=classify model=/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.onnx imgsz=320 data=/content/dataset/IMG_CLASSES  
Visualize:       https://netron.app


'/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.onnx'

In [ ]:
import onnx

# Load the ONNX model
onnx_model = onnx.load("/content/drive/MyDrive/YOLOALGO1/runs/classify/train/weights/best.onnx")  # Load the model object

# Prepare for TensorFlow conversion
tf_rep = prepare(onnx_model)


In [ ]:
tf_rep.export_graph("saved_model")


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]


In [ ]:
tflite_model = converter.convert()

# Save the TFLite model
open("converted_model.tflite", "wb").write(tflite_model)


144845368

In [ ]:
!scp -r /content/converted_model.tflite '/content/drive/MyDrive'

In [ ]:
# Step 2: Data Preparation
!unzip -q '/content/drive/MyDrive/IMG_CLASSES.zip' -d '/content/dataset'

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil

# Define class names and the number of images per class
class_names = [
    "Atopic Dermatitis", "Basal Cell Carcinoma (BCC)", "Benign Keratosis-like Lesions (BKL)", "Eczema",
    "Melanocytic Nevi (NV)", "Melanoma", "Psoriasis pictures Lichen Planus and related diseases",
    "Seborrheic Keratoses and other Benign Tumors", "Tinea Ringworm Candidiasis and other Fungal Infections",
    "Warts Molluscum and other Viral Infections"
]

class_image_counts = [1989, 2495, 2079, 2504, 2527, 2516, 2512, 2363, 2498, 2103]

# Organize the data into training and testing sets
base_dir = '/content/dataset/IMG_CLASSES'  # Update to your dataset directory

# Create train and test directories
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Move images into train and test directories
for class_name, image_count in zip(class_names, class_image_counts):
    class_dir = os.path.join(base_dir, class_name)
    images = os.listdir(class_dir)

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=0.3, random_state=42)

    # Create class-specific directories in the train and test directories
    train_class_dir = os.path.join(train_dir, class_name)
    test_class_dir = os.path.join(test_dir, class_name)

    if not os.path.exists(train_class_dir):
        os.makedirs(train_class_dir)
    if not os.path.exists(test_class_dir):
        os.makedirs(test_class_dir)

    # Move images to their respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(train_class_dir, image)
        shutil.move(src, dst)

    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(test_class_dir, image)
        shutil.move(src, dst)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
# Create an ImageDataGenerator with augmentation configurations
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Iterate through each class in the training directory
for class_name in class_names:
    class_dir = os.path.join(train_dir, class_name)
    images = os.listdir(class_dir)

    # Apply augmentation to each image in the class directory
    for image_name in images:
        image_path = os.path.join(class_dir, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.reshape((1,) + img.shape)  # Reshape for flow method
        save_prefix = 'augmented_' + image_name.split('.')[0]  # Prefix for augmented images

        # Generate augmented images
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=class_dir, save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i >= 3:  # Generate 3 augmented versions of each image
                break

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.1.37 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 36.7/201.2 GB disk)


80/20

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=40, imgsz=320)

YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs


100%|██████████| 71.7M/71.7M [00:00<00:00, 290MB/s]


YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs
Transferred 302/302 items from pretrained weights
Ultralytics YOLOv8.1.20 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.yaml, data=/content/dataset/IMG_CLASSES, epochs=40, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

100%|██████████| 6.23M/6.23M [00:00<00:00, 288MB/s]


AMP: checks passed ✅


...


      Epoch    GPU_mem       loss  Instances       Size


      39/40       2.5G      0.063          1        320: 100%|██████████| 1180/1180 [06:18<00:00,  3.12it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 148/148 [01:05<00:00,  2.26it/s]

                   all      0.909      0.996



      Epoch    GPU_mem       loss  Instances       Size


      40/40      2.49G    0.05467          1        320: 100%|██████████| 1180/1180 [06:17<00:00,  3.13it/s]
               classes   top1_acc   top5_acc:  52%|█████▏    | 77/148 [00:33<00:30,  2.30it/s]

70/30

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=32, imgsz=320)

YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs


100%|██████████| 71.7M/71.7M [00:00<00:00, 229MB/s]


YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs
Transferred 302/302 items from pretrained weights
Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.yaml, data=/content/dataset/IMG_CLASSES, epochs=32, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

100%|██████████| 6.23M/6.23M [00:00<00:00, 105MB/s]


AMP: checks passed ✅


...

      32/32      2.58G    0.06938         10        320: 100%|██████████| 1032/1032 [05:21<00:00,  3.21it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 222/222 [01:35<00:00,  2.33it/s]

                   all      0.896      0.995



32 epochs completed in 3.929 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train/weights/best.pt, 72.6MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 16506 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 7080 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 222/222 [01:35<00:00,  2.31it/s]


                   all      0.896      0.995
Speed: 0.2ms preprocess, 2.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


75/25

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=32, imgsz=320)

YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs
YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs
Transferred 302/302 items from pretrained weights
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.yaml, data=/content/dataset/IMG_CLASSES, epochs=32, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stre

train: Scanning /content/dataset/IMG_CLASSES/train... 17687 images, 0 corrupt: 100%|██████████| 17687/17687 [00:00<?, ?it/s]
val: Scanning /content/dataset/IMG_CLASSES/test... 5899 images, 0 corrupt: 100%|██████████| 5899/5899 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 50 weight(decay=0.0), 51 weight(decay=0.0005), 51 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 2 dataloader workers
Logging results to runs/classify/train2
Starting training for 32 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 185/185 [01:24<00:00,  2.18it/s]

                   all      0.676      0.984


...

      32/32      3.25G    0.06933          7        320: 100%|██████████| 1106/1106 [05:57<00:00,  3.09it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 185/185 [01:26<00:00,  2.14it/s]

                   all      0.897      0.996



32 epochs completed in 3.959 hours.
Optimizer stripped from runs/classify/train2/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train2/weights/best.pt, 72.6MB

Validating runs/classify/train2/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 17687 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 5899 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 185/185 [01:27<00:00,  2.12it/s]


                   all      0.897      0.996
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2
Results saved to runs/classify/train2


80-20

In [ ]:
from ultralytics import YOLO
DATA_DIR = '/content/dataset/IMG_CLASSES'
# Load a model
model = YOLO('yolov8l-cls.yaml')  # build a new model from YAML
model = YOLO('yolov8l-cls.pt')  # load a pretrained model (recommended for training)
model = YOLO('yolov8l-cls.yaml').load('yolov8l-cls.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data= DATA_DIR, epochs=32, imgsz=416)

YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs


100%|██████████| 71.7M/71.7M [00:01<00:00, 68.6MB/s]


YOLOv8l-cls summary: 183 layers, 37480744 parameters, 37480744 gradients, 100.1 GFLOPs
Transferred 302/302 items from pretrained weights
Ultralytics YOLOv8.1.37 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=classify, mode=train, model=yolov8l-cls.yaml, data=/content/dataset/IMG_CLASSES, epochs=32, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes

100%|██████████| 6.23M/6.23M [00:00<00:00, 123MB/s]


AMP: checks passed ✅


...

      32/32      3.78G    0.06435          1        416: 100%|██████████| 1180/1180 [01:47<00:00, 10.96it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 148/148 [00:17<00:00,  8.51it/s]

                   all      0.912      0.996



32 epochs completed in 1.143 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/train/weights/best.pt, 72.6MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.1.37 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8l-cls summary (fused): 133 layers, 36197386 parameters, 0 gradients, 98.7 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/dataset/IMG_CLASSES/train... found 18865 images in 10 classes ✅ 
val: None...
test: /content/dataset/IMG_CLASSES/test... found 4721 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 148/148 [00:17<00:00,  8.39it/s]


                   all      0.912      0.996
Speed: 0.2ms preprocess, 1.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


In [ ]:
!scp -r /content/runs '/content/drive/MyDrive/YOLOALGO(80-20)secondtime'